In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd
import numpy as np
from nltk import download
from nltk import word_tokenize

# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)

import tensorflow as tf
from keras import layers


In [4]:
download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
df = pd.read_csv(
    '/content/drive/My Drive/dataset/yelp_all.csv',encoding='utf8', engine='python', error_bad_lines=False)

df.head()

<ipython-input-5-f0cebe1d5866>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv(


,text,stars
0,"I enjoyed my meal, everything came out as expe...",3
1,This was our first time visiting NYKL. It was ...,4
2,We had one of the worst experiences that my gr...,1
3,"(3.75) ~ Good Food, but the Service was a litt...",4
4,I rarely give 5 stars but the Polite Society e...,5


In [6]:
df = df.sample(30000)

In [7]:
data = df['text'].map(word_tokenize).values
total_vocabulary = set(word.lower() for review in data for word in review)  # set created from nested comprehension
print('There are {} unique words in the dataset.'.format(len(total_vocabulary)))
print('There are {} unique tweets in the dataset.'.format(len(data)))


df.head()

There are 85753 unique words in the dataset.
There are 30000 unique tweets in the dataset.


,text,stars
49520,Illuminare is another one of those places that...,2
165451,Neither daughter nor I can figure out why this...,2
178141,"No, no, no. Spent $40 for dinner for 2 people ...",1
154829,These pictures are complied from a few visits ...,4
10511,I grabbed my parents and headed down for brunc...,3


In [8]:
from keras.utils import pad_sequences
from keras.layers import Input, Dense, LSTM, Embedding
from keras.layers import Dropout, Activation, Bidirectional, GlobalMaxPool1D, CuDNNLSTM
from keras.models import Sequential
from keras import initializers, regularizers, constraints, optimizers, layers, optimizers
from keras.preprocessing import text, sequence

In [9]:
# set the emotion/sentiment as our target
target = df['stars']

In [10]:
target

49520     2
165451    2
178141    1
154829    4
10511     3
         ..
189318    1
8873      4
83261     1
198224    2
14929     4
Name: stars, Length: 30000, dtype: int64

In [11]:
# use one hot encoding since our target is categorical
y = pd.get_dummies(target).values
#y = target

In [12]:
y

array([[0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [1, 0, 0, 0, 0],
       ...,
       [1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 0, 0, 1, 0]], dtype=uint8)

In [16]:
# use keras to create a Tokenizer object
tokenizer = text.Tokenizer(num_words=40000)  # limit to the num_words most important ones
tokenizer.fit_on_texts(list(df['text']))
tokenized_texts = tokenizer.texts_to_sequences(df['text'])
X = pad_sequences(tokenized_texts, maxlen=200)

In [17]:
X

array([[  16,   68,  667, ...,    5, 4419, 5756],
       [ 413,   81,    1, ...,    3, 1697,  268],
       [  46,   25, 1985, ...,   38,   73,  598],
       ...,
       [   5,  355,   18, ...,  465,    5,  251],
       [1508,    4,  139, ...,  129,   32,  467],
       [ 766,   39, 1563, ...,  260,   30,  144]], dtype=int32)

In [26]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# create my NN model
model = Sequential()

embedding_size = 128
model.add(Embedding(len(total_vocabulary), embedding_size))
model.add(Bidirectional(CuDNNLSTM(128, return_sequences=True)))
model.add(LSTM(128, return_sequences=True))
model.add(GlobalMaxPool1D())
model.add(Dropout(0.5))
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(50, activation='relu'))
model.add(Dense(5, activation='softmax'))  # use 5 because we have 5 categories
opt = optimizers.Adam(learning_rate=0.0025)
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])
model.summary() # check the shape

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, None, 128)         10976384  
                                                                 
 bidirectional_5 (Bidirectio  (None, None, 256)        264192    
 nal)                                                            
                                                                 
 lstm_5 (LSTM)               (None, None, 128)         197120    
                                                                 
 global_max_pooling1d_4 (Glo  (None, 128)              0         
 balMaxPooling1D)                                                
                                                                 
 dropout_8 (Dropout)         (None, 128)               0         
                                                                 
 dense_12 (Dense)            (None, 50)               

In [28]:
model.fit(X_train, y_train, epochs=20, batch_size=64, validation_split=0.1)

Epoch 1/20
338/338 [==============================] - 13s 39ms/step - loss: 1.6071 - accuracy: 0.2211 - val_loss: 1.6099 - val_accuracy: 0.2125
Epoch 2/20
338/338 [==============================] - 13s 37ms/step - loss: 1.6097 - accuracy: 0.2030 - val_loss: 1.5535 - val_accuracy: 0.3183
Epoch 3/20
338/338 [==============================] - 13s 37ms/step - loss: 1.5874 - accuracy: 0.2400 - val_loss: 1.5895 - val_accuracy: 0.2308
Epoch 4/20
338/338 [==============================] - 13s 37ms/step - loss: 1.5762 - accuracy: 0.2500 - val_loss: 1.5831 - val_accuracy: 0.2367
Epoch 5/20
338/338 [==============================] - 13s 37ms/step - loss: 1.5520 - accuracy: 0.2728 - val_loss: 1.5356 - val_accuracy: 0.3029
Epoch 6/20
338/338 [==============================] - 13s 37ms/step - loss: 1.3132 - accuracy: 0.3823 - val_loss: 1.2002 - val_accuracy: 0.4542
Epoch 7/20
338/338 [==============================] - 13s 37ms/step - loss: 1.1295 - accuracy: 0.4604 - val_loss: 1.1438 - val_accuracy:

In [29]:
from sklearn.metrics import classification_report,accuracy_score
y_pred = model.predict(X_test) # get our predictions
y_pred = np.argmax(y_pred, axis=1)
y_test = np.argmax(y_test, axis=1)
acc = accuracy_score(y_test, y_pred)

188/188 [==============================] - 2s 10ms/step


In [30]:
acc

0.6145